In [22]:
# import libraries
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

# from src import drop_column_using_vif_, show_vif_values

import imageio
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [23]:
#Read the data
Dataframe = pd.read_csv(os.path.join("data","Cleaned_data.csv"))

In [24]:
# Create a yeas' list
years = ['2018', '2019', '2020', '2021', '2022']

# store many DataFrames by dict
dataframes = {}

for year in years:
    # select all the columns including relating year，and also with the CSScode
    columns_for_year = [col for col in Dataframe.columns if year in col] + ['CSScode']
    # delect possible repeating columns by list-set-list
    columns_for_year = list(set(columns_for_year))
    """
    # set the CSScode as the index of dataframe
    dataframe_year = Dataframe[columns_for_year]
    dataframe_year.set_index('CSScode', inplace=True)
    """
    # create and save the DataFrame back to dicts
    dataframes[f'Dataframe_{year}'] = Dataframe[columns_for_year]

In [25]:
dataframes['Dataframe_2018']

,2018,Finance_Secure_2018,Environment_2018,CSScode,Employment_2018,Healthy_2018,Safety_2018,Decent_Home_2018,Connections_2018
0,134.141667,89.1,44.7,E09000002,69.0,77.5,98.9,70.6,85.0
1,115.575000,91.5,44.3,E09000003,76.4,82.5,99.4,75.0,91.0
2,130.900000,95.4,41.6,E09000004,75.1,79.4,99.3,75.0,83.0
3,115.991667,92.0,48.5,E09000005,68.0,84.2,99.0,84.1,89.0
4,118.908333,97.4,33.4,E09000006,82.5,76.7,99.3,70.4,83.0
5,109.508333,95.3,54.1,E09000007,81.7,77.0,98.6,100.0,95.0
6,128.241667,91.7,37.4,E09000008,76.6,77.0,99.0,86.1,92.0
7,116.400000,95.2,46.1,E09000009,72.0,76.3,99.1,78.0,93.0
8,125.483333,94.6,32.9,E09000010,66.5,75.2,99.1,84.3,86.0
9,122.325000,88.4,58.5,E09000011,79.2,77.3,98.9,86.8,94.0


**Standarlization**

In [21]:
# 选择需要归一化的列
columns_to_normalize = ['Finance_Secure_2018', 'Environment_2018', 'Employment_2018', 
                        'Healthy_2018', 'Safety_2018', 'Decent_Home_2018', 'Connections_2018']

# 对这些列进行归一化
scaler = StandardScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

Index(['2018', 'Finance_Secure_2018', 'Environment_2018', 'CSScode',
       'Employment_2018', 'Healthy_2018', 'Safety_2018', 'Decent_Home_2018',
       'Connections_2018'],
      dtype='object')

In [ ]:
# 添加一个常数项用于计算 VIF
X = sm.add_constant(df[columns_to_normalize])

# 计算 VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

In [ ]:
# 设置因变量和自变量
X = df[columns_to_normalize]  # 自变量
y = df['2018']                # 因变量

# 添加常数项
X = sm.add_constant(X)

# 构建模型
model = sm.OLS(y, X).fit()

# 查看模型结果
print(model.summary())